In [1]:
import os
from lxml import etree 

In [2]:
doc = etree.parse('../Data/xml_data/provenance.xml')
provenance = doc.findall('recordList/record/provenance/provenance.project.txt')
for item in provenance:
    if item.text == None:
        provenance.remove(item)
    else:
        continue

In [3]:
estate_inventory_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if 'Oudezijds Voorburgwal [no. 246]' in elem:
            print(elem)
            continue

        if '{Estate inventory, Jaspar Losschaert, SAA,' in elem:
            print(elem)
            continue
        if '{ Estate inventory, Jaspar Losschaert, SAA, notary A. Lock, NA 2182, fol. 185, 16 June 1667' in elem:
            print(elem)
            continue
        if '{according to the estate inventory, in the great hall, with pendant, SK-A-4462, nos. 13.5, 13.6, Noch Renée de Chalon met Sijne princesse, beyde in profijl., see Drossaers/Lunsingh Scheurleer I, 1974, p. 555.}' in elem:
            print(elem)
            continue
        if ' his son, the historian and MP' in elem:
            print(elem)
            continue
        if 'Estate inventory' in elem:
            estate_inventory_items.append(elem)
        if 'estate inventory' in elem:
            estate_inventory_items.append(elem)

? Estate inventory, Marten Davidsz van Ceulen (1617-87), Oudezijds Voorburgwal [no. 246], Amsterdam, Op de Saal, (‘Twee grootte schilderijen van Rembrandt, d’eene vande worstelingh Jacobs, ende d’ander vande verloocheninge van Petrus’)
{Estate inventory, Jaspar Losschaert, SAA, notary A. Lock, NA 2182, fol. 185, 16 June 1667
{ Estate inventory, Jaspar Losschaert, SAA, notary A. Lock, NA 2182, fol. 185, 16 June 1667
{Estate inventory, Jaspar Losschaert, SAA, notary A. Lock, HNA 2182, fol. 185, 16 June 1667
{according to the estate inventory, in the great hall, with pendant, SK-A-4462, nos. 13.5, 13.6, Noch Renée de Chalon met Sijne princesse, beyde in profijl., see Drossaers/Lunsingh Scheurleer I, 1974, p. 555.} 
{Estate inventory Lough Fea, 1869. See exh. cat. Amsterdam 1980-81 (note 1), no. 5.} his son, the historian and MP of Monaghan, Evelyn Phillip Shirley (1812-82), 1856
{Estate inventory Lough Fea, 1869. See exh. cat. Amsterdam 1980-81 (note 1), no. 5.} his son, the historian an

In [4]:
#code just for the items mentioning estate inventory

for item in estate_inventory_items:
    #possible parts of the provenance sentence
    name = ''
    name2 = '' # if there's a second owner
    BDdate = '' #birth-death date
    BDdate2 = ''
    extra = ''
    extra2 = ''
    place = ''
    place_extra = ''
    date = '' #date of inventory
    location = '' #location of work in house
    no = '' #no in inventory
    fol = ''
    p = ''
    attribution = ''
    description = ''
    source = ''
    start_source = ''
    #fix possible missing comma at the start
    test_item = item
    if 'estate inventory ' in item:
        item = item.replace('estate inventory', 'estate inventory,').replace('estate inventory of','estate inventory,')
    #saving the original, but fixed, item in case I need it
    start_item = item
    #print('Item:', start_item) 
    #certain or uncertain provenance
    certain_uncertain = item.split('(')
    if '?' in certain_uncertain[0]:
        #provenance data is uncertain
        proper_format = '? '
        readable_sent = 'Possibly part of '
    else:
        #provenance data is certain
        proper_format = ''
        readable_sent = 'Part of '
    
    #source
    #see if item starts with source but a { is missing
    check_split = item.split('estate')
    if '}' in check_split[0] and not '{' in check_split[0]:
        item = '{'+item
    #if item starts with source
    if item.startswith('{') or item.startswith(' {'):
        curly_split = item.split('}')
        start_source = curly_split[0].replace('{', '').strip()
    #if item ends with source
    if '{' in item and start_source == '':
        curly_split = item.split('{')
        source = curly_split[1].replace('}', '').strip()
    if start_source != '':
        sourceless_item = start_item.replace(start_source,'').strip()
    else:
        sourceless_item = start_item.replace(source,'').strip()
    if start_source != '':
        start_source = '{'+start_source+'}'
    if source != '':
        source = '{'+source+'}'
    
    #name owner
    comma_split = sourceless_item.split(',')
    #no owner name, owner name in previous provenance sentence
    if 'his ' in comma_split[0]:
        proper_format = f"{proper_format}his "
        readable_sent = f"{readable_sent}his "
    elif 'her ' in comma_split[0]:
        proper_format = f"{proper_format}her "
        readable_sent = f"{readable_sent}her "
    elif 'their ' in comma_split[0]:
        proper_format = f"{proper_format}their "
        readable_sent = f"{readable_sent}their "
    #owner name is mentioned
    else:
        name_elem = comma_split[1]
        if ' and ' in name_elem:
            two_names = name_elem.split('and')
            name_one = two_names[0]
            name = name_one.strip()
            if '(' in name_one:
                name_one = name_one.split('(')
                name = name_one[0].strip()
                #BDdate person 1
                BDdate = name_one[1].replace(')','').strip()
            name_two = two_names[1]
            name2 = name_two.strip()
            if '(' in name_two:
                name_two = name_two.split('(')
                name2 = name_two[0].strip()
                #BDdate person 2
                BDdate2 = name_two[1].replace(')','').strip()
        else:
            name = name_elem.strip()
            name_elem = name_elem.split('(')
            name = name_elem[0].strip()
            if 'House' in name or 'house' in name or 'Huis' in name or 'huis' in name or 'Castle' in name or 'castle' in name or 'Kasteel' in name or 'kasteel' in name or 'Paleis' in name or 'paleis' in name or 'palace' in name or 'Palace' in name:
                place = name
                name = ''
            if len(name_elem) > 1 and ('1' in name_elem[1] or '2' in name_elem[1]) and name !='':
                BDdate = name_elem[1].replace(')','').strip()
            if ('1' in comma_split[2] or '2' in comma_split[2]) and BDdate == '' and name !='':
                extra = comma_split[2]
                extra_elem = extra.split('(')
                extra = extra_elem[0].strip()
                BDdate = extra_elem[1].replace(')','').strip()
            if '1' in name_elem or '2' in name_elem and BDdate == '' and name !='':
                BDdate = name_elem[1].replace(')','').strip()
                
        #extra info owner
        if BDdate not in comma_split[1]:
            extra = comma_split[2]
            extra = extra.split('(')
            extra = extra[0].strip()
        #if extra info is about being a widow
        if ' widow ' in comma_split[2]:
            extra = comma_split[2].strip()
                
    #inventory date
    for item in comma_split:
        #no in inventory
        if 'no.' in item:
            no = item
            if '(' in item:
                split_item = item.split('(')
                no = split_item[0].strip()
        if 'fol.' in item:
            fol = item.strip()
        if 'p.' in item:
            p = item.strip()
        #actual inventory date
        item = item.split('(')
        if ('1' in item[0] or '2' in item[0]) and date == '' and not "'" in item[0] and not "‘" in item[0]:
            date = item[0]
            
    #description
    para_split = sourceless_item.split('(')
    for item in para_split:
        if "')" in item or "’)" in item:
            item = item.split(')')
            description = item[0].strip()  
        
            
    #attribution
    for item in comma_split:
        if 'without attribution' in item:
            attribution = 'without attribution'
        if ' as ' in item:
            attribution = item
            attribution = attribution.split('(')
            attribution = attribution[0].strip()
        if "'" in item or "‘" in item:
            item = item.split('(')
            if "'" in item[0] or "‘" in item[0]:
                attribution = item[0].strip()
            
    #location in the house
    for item in comma_split:
        if 'in the ' in item:
            item = item.split("(")
            location = item[0].strip()
        if 'Op de ' in item or 'op de ' in item:
            item = item.split('(')
            location = item[0].strip()
        if 'chamber' in item:
            item = item.split("(")
            location = item[0].strip()
            if "'" in location or "‘" in location:
                location = ''
    
    #place
    #index number of date
    for item in comma_split:
        if date in item and not "'" in item and not "‘" in item:
            date_index = comma_split.index(item)
            place = comma_split[(date_index - 1)]
    if 'near ' in place:
        place_extra = place
        place = comma_split[(date_index - 2)].strip()
    if 'estate inventory' in place:
        place = ''
    if place != '' and place_extra == '':
        place_index = comma_split.index(place)
        if BDdate not in comma_split[(place_index-1)] and BDdate2 not in comma_split[(place_index-1)] and name not in comma_split[(place_index-1)] and name2 not in comma_split[(place_index-1)]:
            place_extra = place
            place = comma_split[(place_index-1)].strip() 
        if 'near ' in comma_split[(place_index+1)]:
            place_extra = comma_split[(place_index+1)]
    #extra info owner 2
    if place != '':
        place_index = ' ' + place
        if '  ' in place_index:
            place_index = place_index.replace('  ',' ')
        place_index = comma_split.index(place_index)
        index = (place_index - 1)
        if name2 not in comma_split[index] and extra not in comma_split[index] and name not in comma_split[index]:
            extra2 = comma_split[(place_index -1)].strip()
    
    if name == 'William III':
        extra = 'Prince of Orange, King of England'
    if name == 'James II':
        extra = 'King of England'
    if date in place:
        place = ''
    
    #check just in case if everything is properly stripped
    start_source = start_source.strip()
    name = name.strip()
    BDdate = BDdate.strip()
    extra = extra.strip()
    name2 = name2.strip()
    BDdate2 = BDdate2.strip()
    extra2 = extra2.strip()
    place = place.strip()
    place_extra = place_extra.strip()
    date = date.strip()
    location = location.strip()
    fol = fol.strip()
    p = p.strip()
    no = no.strip()
    attribution = attribution.strip()
    description = description.strip()
    source = source.strip()
    
    #proper format and readable sentence
    proper_format = f"{proper_format}estate inventory"
    readable_sent = f"{readable_sent}estate inventory"
    #proper format
    if start_source != '':
        proper_format = f"{start_source} {proper_format}"
    if name != '':
        proper_format = f"{proper_format}, {name}"
    if BDdate != '':
        proper_format = f"{proper_format} ({BDdate})"
    if extra != '':
        proper_format = f"{proper_format}, {extra}"
    if name2 != '' and extra != '':
        proper_format = f"{proper_format}, and {name2}"
    if name2 != '' and extra == '':
        proper_format = f"{proper_format} and {name2}"
    if BDdate2 != '':
        proper_format = f"{proper_format} ({BDdate2})"
    if extra2 != '':
        proper_format = f"{proper_format}, {extra2}"
    if place != '':
        proper_format = f"{proper_format}, {place}"
    if place_extra != '':
        proper_format = f"{proper_format}, {place_extra}"
    if date != '':
        proper_format = f"{proper_format}, {date}"
    if location != '':
        proper_format = f"{proper_format}, {location}"
    if fol != '':
        proper_format = f"{proper_format}, {fol}"
    if p != '':
        proper_format = f"{proper_format}, {p}"
    if no != '':
        proper_format = f"{proper_format}, {no}"
    if attribution != '':
        proper_format = f"{proper_format}, {attribution}"
    if description != '':
        proper_format = f"{proper_format} ({description})"
    if source != '':
        proper_format = f"{proper_format} {source}"
    #print('Proper format:', proper_format)
    
    #readable sentence
    if name != '':
        readable_sent = f"{readable_sent} of {name}"
    if BDdate != '':
        readable_sent = f"{readable_sent} ({BDdate})"
    if extra != '':
        readable_sent = f"{readable_sent}, {extra}"
    if name2 != '' and extra != '':
        readable_sent = f"{readable_sent}, and {name2}"
    if name2 != '' and extra == '':
        readable_sent = f"{readable_sent} and {name2}"
    if BDdate2 != '':
        readable_sent = f"{readable_sent} ({BDdate2})"
    if extra2 != '':
        readable_sent = f"{readable_sent}, {extra2}"
    if place != '':
        readable_sent = f"{readable_sent}, {place}"
    if place_extra != '':
        readable_sent = f"{readable_sent}, {place_extra}"
    if date != '':
        readable_sent = f"{readable_sent}, {date}"
    if location != '':
        readable_sent = f"{readable_sent}, hanging {location}"
    if fol != '':
        readable_sent = f"{readable_sent}, {fol}"
    if p != '':
        readable_sent = f"{readable_sent}, {p}"
    if no != '':
        readable_sent = f"{readable_sent}, {no}"
    if attribution != '':
        readable_sent = f"{readable_sent}, {attribution}"
    if description != '':
        readable_sent = f"{readable_sent} ({description})"
    readable_sent = f"{readable_sent}."
    
    #test where mistakes are made
    if proper_format.strip() != test_item.strip():
        print(test_item)
        print('Proper format:', proper_format)
        print('\n')
    

 ? estate inventory, Anna van Renesse, Lady of Assendelft (1622-67), Kasteel Assumburgh, near Heemstede, 1667, ‘Old gentleman’s room above the great chamber’ (‘een oud schilderije’)
Proper format: ? estate inventory, Anna van Renesse (1622-67), Lady of Assendelft, Kasteel Assumburgh, near Heemstede, 1667, ‘Old gentleman’s room above the great chamber’ (‘een oud schilderije’)


 ? estate inventory Jan Six (1618-1700) and Margareta Tulp (1634-1709), Herengracht, Amsterdam, 6 January 1710, in the art room (‘Een avondtstondt van Aart van der Neer met beelden etc. [fl] 15’)
Proper format: ? estate inventory, Jan Six (1618-1700) and Margareta Tulp (1634-1709), Herengracht, Amsterdam, 6 January 1710, in the art room (‘Een avondtstondt van Aart van der Neer met beelden etc. [fl] 15’)


 Strauss et al. 1979, p. 461, doc. no. 1660/15 } ? estate inventory of her son, Tobias van Domselaer (1611-85), Amsterdam, September 1685 (‘een groot schilderij met twee paeuwen van Rembrandt’)
Proper format: {S